In [1]:
from datetime import datetime

In [2]:
BATCH_SIZE = 32
n_seeds = 1
n_splits = 10
n_epochs = 1

RESULT_TXT = f"roberta-{datetime.now().strftime('%Y%m%d-%H%M%S')}.txt"
RESULT_PATH = "../models/fine-tune-roberta"

OUT_DROPOUT = 0.3

TRAIN_ON_N_SPLITS = 1

SUBGROUP_NEGATIVE_WEIGHT_COEF = 1

BERT_N_LAST_LAYER = 4
BERT_HIDDEN_SIZE = 768

BERT_MODEL_PATH = 'roberta-base'
BERT_DO_LOWER = False

MAX_LEN = 220

In [3]:
RESULT_TXT

'roberta-20220203-234253.txt'

In [4]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm import tqdm
tqdm.pandas()

import torch
from transformers import RobertaTokenizer, RobertaModel

from contextlib import contextmanager
from fastprogress import master_bar, progress_bar
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
if not os.path.exists(RESULT_PATH):
    os.mkdir(RESULT_PATH)

In [7]:
# 9.9G

train = pd.read_csv('../../data/all_data.csv').rename(columns={'toxicity': 'target'})

train = train[train['comment_text'].str.len() > 1].reset_index(drop=True)

y_train = np.where(train['target'] >= 0.5, 1, 0)
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

In [8]:
# import joblib

# puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&',
#           '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
#           '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›', 
#           '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', 
#           '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯',
#           '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔',
#           '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '\n', '\r']

# with open('../../jigsaw-crawl-300d-2M.joblib', 'rb') as f:
#     crawl_emb_dict = joblib.load(f)

In [9]:
# with open('../../google-profanity-words/profanity.js', 'r') as handle:
#     p_words = handle.readlines()
    
# set_puncts = set(puncts)

# p_word_set = set([t.replace('\n', '') for t in p_words])

# def sentence_fetures(text):
#     word_list = text.split()
#     word_count = len(word_list)
#     n_upper = len([word for word in word_list if any([c.isupper() for c in word])])
#     n_unique = len(set(word_list))
#     n_ex = word_list.count('!')
#     n_que = word_list.count('?')
#     n_puncts = len([word for word in word_list if word in set_puncts])
#     n_prof = len([word for word in word_list if word in p_word_set])
#     n_oov = len([word for word in word_list if word not in crawl_emb_dict])
    
#     return word_count, n_upper, n_unique, n_ex, n_que, n_puncts, n_prof, n_oov

# sentence_feature_cols = ['word_count', 'n_upper', 'n_unique', 'n_ex', 'n_que', 'n_puncts', 'n_prof', 'n_oov']

In [10]:
# from collections import defaultdict

# feature_dict = defaultdict(list)

# for text in tqdm(train['comment_text']):
#     feature_list = sentence_fetures(text)
#     for i_feature, feature_name in enumerate(sentence_feature_cols):
#         feature_dict[sentence_feature_cols[i_feature]].append(feature_list[i_feature])
        
# sentence_df = pd.DataFrame.from_dict(feature_dict)

# for col in ['n_upper', 'n_unique', 'n_ex', 'n_que', 'n_puncts', 'n_prof', 'n_oov']:
#     sentence_df[col + '_ratio'] = sentence_df[col] / sentence_df['word_count']

In [11]:
# sentence_df = pd.read_csv('../input/sentencefeaturesoov/sentence_features.csv')

In [12]:
# sentence_df.to_parquet('sent_df.parquet')

In [13]:
sentence_df = pd.read_parquet('sent_df.parquet')

In [14]:
sentence_df.head()

,word_count,n_upper,n_unique,n_ex,n_que,n_puncts,n_prof,n_oov,n_upper_ratio,n_unique_ratio,n_ex_ratio,n_que_ratio,n_puncts_ratio,n_prof_ratio,n_oov_ratio
0,36,2,33,0,0,0,0,3,0.055556,0.916667,0.0,0.0,0.0,0.000000,0.083333
1,11,2,11,0,0,0,0,0,0.181818,1.000000,0.0,0.0,0.0,0.000000,0.000000
2,18,3,18,0,0,0,0,0,0.166667,1.000000,0.0,0.0,0.0,0.000000,0.000000
3,24,2,22,0,0,0,0,4,0.083333,0.916667,0.0,0.0,0.0,0.000000,0.166667
4,9,2,9,0,0,0,1,1,0.222222,1.000000,0.0,0.0,0.0,0.111111,0.111111


In [15]:
sentence_feature_mat = sentence_df.values
del sentence_df
gc.collect()

61

In [16]:
TOXICITY_COLUMN = 'target'
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 
    'christian', 'jewish', 'muslim', 'black', 
    'white', 'psychiatric_or_mental_illness'
]

subgroup_bool_train = (train[identity_columns].fillna(0) >= 0.5)
toxic_bool_train = (train[TOXICITY_COLUMN].fillna(0) >= 0.5)
subgroup_negative_mask = (
    subgroup_bool_train.values.sum(axis=1).astype(bool) & (~toxic_bool_train)
)

sample_weight = np.ones((y_train.shape[0],))
sample_weight += SUBGROUP_NEGATIVE_WEIGHT_COEF * subgroup_negative_mask

In [17]:
stacked_target = np.hstack([
    np.expand_dims(y_train, axis=1), 
    y_aux_train,
    np.expand_dims(sample_weight, axis=1)
])

In [18]:
BERT_MODEL_PATH = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(BERT_MODEL_PATH)

In [19]:
def tokenize(row, max_len, tokenizer):
    tokenized = tokenizer.encode_plus(
        row,
        truncation=True,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=False
    )
    return tokenized['input_ids'], tokenized['attention_mask']

In [20]:
# from pandarallel import pandarallel

# pandarallel.initialize()

In [21]:
# df_x_tokenized = train['comment_text'].parallel_apply(
#     lambda x: tokenize(x, MAX_LEN, tokenizer)
# )
# x_train_indexed = df_x_tokenized
# pd.DataFrame(x_train_indexed).to_parquet('x_train_tokenized_roberta.parquet')

In [22]:
x_train_indexed = pd.read_parquet('x_train_tokenized_roberta.parquet')['comment_text']
x_train_indexed = x_train_indexed.reset_index(drop=True)

In [23]:
from torch.utils.data import Dataset, random_split, DataLoader

class ReviewsDataset(Dataset):
    def __init__(self, tokenized, sent_features, labels):
        self.labels = labels
        self.sent_features = sent_features
        self.tokenized = tokenized
        
    def __getitem__(self, idx):
        return {
            "ids": torch.tensor(self.tokenized[idx][0], dtype=torch.long),
            "attention": torch.tensor(self.tokenized[idx][1], dtype=torch.long),
            "sent_features": torch.tensor(self.sent_features[idx], dtype=torch.float),
            "labels": torch.tensor(self.labels[idx][:-1], dtype=torch.float),
            "sample_weight": torch.tensor(self.labels[idx][-1], dtype=torch.float)
        }

    def __len__(self):
        return len(self.labels)

In [24]:
class NeuralNet(nn.Module):
    def __init__(self, num_aux_targets, num_sentence_features):
        super(NeuralNet, self).__init__()
        self.bert_model = RobertaModel.from_pretrained(BERT_MODEL_PATH)
        self.dropout = nn.Dropout(OUT_DROPOUT)
        
        self.linear_sentence1 = nn.Linear(num_sentence_features, num_sentence_features)
        
        n_hidden = BERT_HIDDEN_SIZE + num_sentence_features
        self.linear1 = nn.Linear(n_hidden, n_hidden)
        
        self.linear_out = nn.Linear(n_hidden, 1)
        self.linear_aux_out = nn.Linear(n_hidden, num_aux_targets)
        
    def forward(self, ids, attention, sentence_features):
        
        bert_output = self.bert_model(ids, attention)[1]
        
        bert_output = self.dropout(bert_output)
        
        h_sentence = self.linear_sentence1(sentence_features)
        
        h_cat = torch.cat((bert_output, h_sentence), 1)
        
        h_conc_linear1  = F.relu(self.linear1(h_cat))
        
        hidden = h_cat + h_conc_linear1
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [25]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
        self.old_time = time.time()
        print('start mearsure elapsed times')
        
    def stamp(self, comment):
        print(comment + f': from start {time.time() - self.start_time: .1f}, from old {self.old_time - - self.start_time: .1f}')
        self.old_time = time.time()

In [26]:
def adjust_lr(optimizer, i_batch, min_lr, max_lr, n_batch_all, warm_up_batch_ratio):
    n_batch_warmed = int(n_batch_all * warm_up_batch_ratio)
    if i_batch > n_batch_warmed:
        optimizer.param_groups[0]['lr'] = max_lr
    else:
        optimizer.param_groups[0]['lr'] = (max_lr - min_lr) / n_batch_warmed * i_batch + min_lr

In [27]:
import joblib
from sklearn.preprocessing import StandardScaler

timer = ElapsedTimer()
loss_fn=nn.BCEWithLogitsLoss(reduction='sum')

checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
checkpoint_weights = np.array(checkpoint_weights) / np.sum(checkpoint_weights)

dev_loss_array = np.zeros((n_seeds, TRAIN_ON_N_SPLITS, n_epochs))
val_loss_array = np.zeros((n_seeds, TRAIN_ON_N_SPLITS, n_epochs))

auc_array = np.zeros((n_seeds, TRAIN_ON_N_SPLITS, n_epochs))

seed_oof_train_epoch_weighted_list = []
seed_oof_train_last_list = []

oof_train = np.zeros((n_seeds, n_epochs, len(x_train_indexed)))

from sklearn.model_selection import KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1999)

for i_seed in range(n_seeds):
    print(f'start seed {i_seed}')
    timer.stamp('seed start')
    fold_dev_loss_list = []
    fold_val_loss_list = []
    oof_train_epoch_weighted = np.zeros(len(x_train_indexed))
    oof_train_last = np.zeros(len(x_train_indexed))
    
    for i_fold, (dev_index, val_index) in enumerate(kf.split(x_train_indexed)):
        
        if i_fold > TRAIN_ON_N_SPLITS:
            continue
        timer.stamp('epoch start')
            
        print(f'start fold {i_fold}')
  
        # Load pre-trained model (weights)
        model = NeuralNet(y_aux_train.shape[-1], sentence_feature_mat.shape[-1]).to(device)
        print('model', torch.cuda.memory_allocated())

        optimizer = torch.optim.Adam(model.parameters())
        
        scaler = StandardScaler()
        dev_sentence_feature_mat = scaler.fit_transform(sentence_feature_mat[dev_index])
        val_sentence_feature_mat = scaler.transform(sentence_feature_mat[val_index])
        
        joblib.dump(scaler, os.path.join(RESULT_PATH, f'scaler-seed{i_seed}-fold{i_fold}.joblib'))
        
        dev_ds = ReviewsDataset(
            x_train_indexed[dev_index].reset_index(drop=True), 
            dev_sentence_feature_mat, 
            stacked_target[dev_index]
        )
        dev_loader = DataLoader(
            dev_ds, batch_size=BATCH_SIZE, shuffle=True,
        )
        
        valid_ds = ReviewsDataset(
            x_train_indexed[val_index].reset_index(drop=True), 
            val_sentence_feature_mat, 
            stacked_target[val_index]
        )
        valid_loader = DataLoader(
            dev_ds, batch_size=BATCH_SIZE, shuffle=False,
        )
        
        all_test_preds = []
        dev_loss_list = []
        val_loss_list = []
        weighted_val_pred = np.zeros(val_index.shape[0])

        for i_epoch in range(n_epochs):
            timer.stamp(f'i_epoch {i_epoch} start')
            
            print(f'epoch_start: {i_epoch}', torch.cuda.memory_allocated())
            start_time = time.time()
            
            model.train()
            dev_avg_loss = []
            for i_batch, batch in enumerate(progress_bar(dev_loader)):
                if i_epoch == 0:
                    adjust_lr(
                        optimizer, i_batch, min_lr=1e-6, max_lr=1e-5,
                        n_batch_all=len(dev_loader), warm_up_batch_ratio=0.1
                    ) 
                                
                y_pred = model(
                    batch['ids'].to(device), 
                    batch['attention'].to(device), 
                    batch['sent_features'].to(device)
                )
                
                loss_fn = nn.BCEWithLogitsLoss(
                    batch['sample_weight'][:, None].to(device), reduction='sum'
                )
                loss = loss_fn(y_pred, batch['labels'].to(device))

                optimizer.zero_grad()
                loss.backward()

                optimizer.step()
                dev_avg_loss += [loss.item()]
                if i_batch % 100 == 0:
                    print(np.hstack(dev_avg_loss).mean())
            
            optimizer.zero_grad()
            timer.stamp(f'after dev all batch')
            gc.collect()
            torch.cuda.empty_cache()
            
            torch.save(model.state_dict(), os.path.join(RESULT_PATH, 
                f'seed{i_seed}-fold{i_fold}-epoch{i_epoch}.torchModelState'))

start mearsure elapsed times
start seed 0
seed start: from start  0.0, from old  3287841997.7
epoch start: from start  0.2, from old  3287841997.7
start fold 0


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model 502366208
i_epoch 0 start: from start  5.0, from old  3287841997.9
epoch_start: 0 502366208


178.84466552734375
163.30393709994777
126.8285970165955
101.47682644916927
87.11897114684754
78.44788923282586
72.27723314599467
67.46863379866183
63.60877637113078
60.36107162153814
57.80496781165307
55.553485984698305
53.63188196717452
52.00656595750555
50.498793997482096
49.26874797650768
48.0881362220483
47.04699115315863
46.048747523369755
45.19742174665279
44.46665450109952
43.75192298280915
43.06552329663524
42.356621949478935
41.77575748813793
41.24536102040202
40.75728449221988
40.32974110193405
39.91799664965531
39.51310605618182
39.13294725503893
38.78931291008795
38.425390859053906
38.10369944984138
37.80538531788235
37.5160671496725
37.248755591407615
36.98626078930587
36.7164415245588
36.43253567914906
36.24234705381291
36.02393453349662
35.84743369735158
35.62750192443539
35.41937977117561
35.266895877390645
35.095686321076144
34.95444351053065
34.794855106177366
34.63792223629721
34.46639003049991
34.33150405777223
34.199866066848145
34.06108063136693
33.92017096825119


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model 2516131328
i_epoch 0 start: from start  12020.2, from old  3287854015.1
epoch_start: 0 1510761984


162.58250427246094
147.79249799369586
111.80507523266238
91.64158809066215
79.86168566011729
72.44980693291761
67.6655652050964
63.73287893940141
60.44011179844241
57.33376490712563
54.89059815040002
52.86539552083998
51.179980125554295
49.63631510313065
48.29144381932239
47.15569990400788
46.04413257696567
45.09930628149177
44.2018255581133
43.37381924233394
42.65627641835134
42.01604485500432
41.4227203792033
40.870921666701946
40.35713836283845
39.90331878646857
39.452887281002056
39.049944362477966
38.68893464009109
38.283899219148694
37.91657952172325
37.6369061276283
37.339967135227084
37.038262131357875
36.78340983327716
36.529914940265
36.25852714224479
36.00299051226039
35.81991460466975
35.58542613535776
35.396642493176714
35.22660502385175
35.02243122842249
34.831324227990955
34.63548297823572
34.45506798090126
34.33366821973486
34.15716114675104
34.03672423806694
33.88986571324988
33.742539997650034
33.60841492619428
33.4922710669726
33.378156485590836
33.25210447555073
33.